In [2]:
import pysrt
from pysrt.srttime import SubRipTime
import pandas as pd
from bs4 import BeautifulSoup
from IPython.display import clear_output
import chardet

In [57]:
heb_sub_path = r"Data/Hebrew/Pirates.of.the.Caribbean.Dead.Men.Tell.No.Tales.2017.srt"
eng_sub_path = r"Data/English/Pirates.of.the.Caribbean.Dead.Men.Tell.No.Tales.2017.srt"

In [58]:
# Find encoding
with open(heb_sub_path, 'rb') as file:
    result = chardet.detect(file.read())
encoding = result['encoding']
encoding

'utf-8'

In [59]:
heb_subs = pysrt.open(heb_sub_path, encoding=encoding)
eng_subs = pysrt.open(eng_sub_path)

In [40]:
# heb_subs.shift(seconds=5)

In [6]:
dif = 0.8
dif = int(dif * 1000)

In [60]:
subs = pd.DataFrame(columns=["Hebrew", "English", "Safe", "Hebrew_st", "Hebrew_e", "English_st", "English_e"])
for heb_sub in heb_subs:
    h_start = heb_sub.start
    h_end = heb_sub.end
    safe = True
    for eng_sub in eng_subs:
        e_start = eng_sub.start
        e_end = eng_sub.end

        if h_start + 2000 < e_start:
            break

        if h_start - dif > e_start:  # More than (dif) second apart
            continue
        if (e_end - dif) < h_end < (e_end + dif):
            if (e_start - dif) < h_start < (e_start + dif):
                heb_text = BeautifulSoup(heb_sub.text, "html.parser").getText()  # Removes html format like <i>
                eng_text = BeautifulSoup(eng_sub.text, "html.parser").getText()

                if heb_text.count("\n") != eng_text.count("\n"):  # Check if the numbers of lines is not the same - meaning chance of unmatching sentences
                    safe = False
                if heb_text.count("?") != eng_text.count("?"):  # Check if the numbers of lines is not the same - meaning chance of unmatching sentences
                    safe = False
                subs.loc[subs.shape[0]] = [heb_text, eng_text, safe, h_start, h_end, e_start, e_end]
                break



In [61]:
subs[subs["Safe"] != True]

,Hebrew,English,Safe,Hebrew_st,Hebrew_e,English_st,English_e
2,".""אין לך מקום ב""ההולנדי",There's no place for you\non the Dutchman.,False,"00:03:47,090","00:03:49,300","00:03:47,020","00:03:49,772"
13,.הוא יעזור לי למצוא את הקלשון,He will help me\nfind the Trident.,False,"00:04:37,480","00:04:38,769","00:04:37,153","00:04:38,613"
14,...אתה חייב להפסיק להתנהג כמו,You have to stop\nacting like...,False,"00:04:42,980","00:04:43,939","00:04:42,659","00:04:43,660"
18,.הקללה שלי לא תישבר לעולם,But my curse\nwill never be broken.,False,"00:05:01,710","00:05:03,460","00:05:01,427","00:05:03,429"
30,!אני צריך לדבר עם הקפטן\n?מה אמרת-,I need to speak\nwith the captain.,False,"00:06:41,350","00:06:43,229","00:06:41,360","00:06:42,487"
...,...,...,...,...,...,...,...
816,"!הנרי\n.לפצל, לשבור-",Divide. Break.,False,"01:47:07,080","01:47:09,039","01:47:07,631","01:47:08,799"
835,".לרגע אחד היה לי הכל, הנרי","For a moment,\nI had everything, Henry.",False,"01:53:45,310","01:53:47,770","01:53:44,944","01:53:47,780"
844,"?כן, אבא","Yes, Father.",False,"01:56:16,590","01:56:17,590","01:56:16,387","01:56:17,472"
854,.קפטן\n.תודה-,Thank you.,False,"01:57:58,610","01:58:00,569","01:57:59,407","01:58:00,241"


In [9]:
unsafe_len = subs[subs["Safe"] != True].shape[0]
i = 0
for index, row in subs[subs["Safe"] != True].iterrows():
    i += 1
    while True:
        print(f"{i}\\{unsafe_len}")
        print(row["Hebrew_st"],"-",row["Hebrew_e"])
        print(row['Hebrew'])
        print(row['English'])
        res = input()
        if res == "x":
            clear_output()
            print(f"Marking unsafe: {subs.iloc[index]['Hebrew']}\n")
            break
        elif res == "v":
            clear_output()
            print(f"Marking safe: {subs.iloc[index]['Hebrew']}\n")
            subs.loc[index, "Safe"] = True
            break
        elif res == "break":
            break
        else:
            clear_output()
            print("Invalid input, x-delete, v-save, break-break\n")
            continue
    if res == "break":
        break

Invalid input, x-delete, v-save, break-break

1\34
00:03:55,199 - 00:03:56,399
.אולי הבסיס קרוב
And our donkey
just spotted another trench.


In [62]:
subs = subs[subs["Safe"] == True]

In [63]:
subs["Hebrew"] = subs["Hebrew"].str.replace("\n", ", ")
subs["English"] = subs["English"].str.replace("\n", ", ")
subs["Hebrew"] = subs["Hebrew"].str.replace(":", ", ")
subs["English"] = subs["English"].str.replace(":", ", ")
subs["Hebrew"] = subs["Hebrew"].str.replace('"', "")
subs["English"] = subs["English"].str.replace('"', "")

In [64]:
subs.to_csv(r"Data/Modified/Pirates.of.the.Caribbean.Dead.Men.Tell.No.Tales.2017.csv", index=False)

In [65]:
test = pd.read_csv(r"Data/Modified/Pirates.of.the.Caribbean.Dead.Men.Tell.No.Tales.2017.csv")
test

,Hebrew,English,Safe,Hebrew_st,Hebrew_e,English_st,English_e
0,".תסתכל עליי, בן","Look at me, son.",True,"00:03:41,340","00:03:42,760","00:03:41,180","00:03:42,932"
1,.לא אכפת לי,I don't care.,True,"00:03:45,720","00:03:46,880","00:03:45,518","00:03:46,853"
2,.הם יודעים שאתה פה,They know you're here.,True,"00:03:53,060","00:03:54,350","00:03:52,859","00:03:54,444"
3,",לא, בחיים לא אפסיק",I won't! I'll never stop.,True,"00:04:00,270","00:04:02,730","00:04:00,283","00:04:02,535"
4,"ואם תשליך אותי מהספינה, .אחזור ישר","And if you throw me over,, I'll come straight ...",True,"00:04:02,980","00:04:06,070","00:04:02,702","00:04:05,914"
...,...,...,...,...,...,...,...
678,.תודה,Thank you.,True,"01:58:28,470","01:58:29,640","01:58:28,269","01:58:29,604"
679,"?לאן נשוט, קפטן","What be our heading, Captain?",True,"01:58:30,600","01:58:32,719","01:58:30,313","01:58:32,648"
680,",נעקוב אחר הכוכבים, .רב חובל גיבס","We shall follow the stars,, Master Gibbs.",True,"01:58:32,720","01:58:35,730","01:58:32,815","01:58:35,610"
681,".כן, קפטן","Aye, aye, Captain!",True,"01:58:36,140","01:58:38,100","01:58:35,777","01:58:38,404"


In [82]:
# Loads back the SubRipTime type
test['Hebrew_st'] = test['Hebrew_st'].apply(lambda x: SubRipTime.from_string(x) if isinstance(x, str) else x)
test['Hebrew_e'] = test['Hebrew_e'].apply(lambda x: SubRipTime.from_string(x) if isinstance(x, str) else x)
test['English_st'] = test['English_st'].apply(lambda x: SubRipTime.from_string(x) if isinstance(x, str) else x)
test['English_e'] = test['English_e'].apply(lambda x: SubRipTime.from_string(x) if isinstance(x, str) else x)

In [122]:
subs

,Hebrew,English,Safe,Hebrew_st,Hebrew_e,English_st,English_e
0,!גול,Goal!,True,"00:02:11,928","00:02:13,367","00:02:12,345","00:02:13,835"
2,",מה שזה לא יהיה, .זה רחוק מכאן","I heard it, Manny., Whatever it is, it's miles...",True,"00:03:27,611","00:03:29,011","00:03:26,887","00:03:29,219"
3,"?פיצ'ס, את בסדר","Peaches, are you all right?",True,"00:03:29,118","00:03:31,076","00:03:29,389","00:03:30,720"
5,"שניכם הייתם אמורים להיות, !דודים אחראיים","You two were supposed to be, responsible uncles!",True,"00:03:42,116","00:03:44,616","00:03:42,469","00:03:44,869"
6,"מה? לא ראיתי את פיצ'ס מתגנבת, .לפני 15 או 20 דקות","What? I didn't see Peaches sneak off, maybe 15...",True,"00:03:44,728","00:03:48,113","00:03:44,938","00:03:48,169"
...,...,...,...,...,...,...,...
395,- וונדה סייקס -,# Somehow we found it here #,True,"01:20:39,405","01:20:42,781","01:20:39,882","01:20:42,942"
396,- ג'ניפר לופז -,# We found us a home #,True,"01:20:43,028","01:20:46,468","01:20:43,118","01:20:46,576"
399,"- אלאיין צ'באט -, - הת'ר מוריס -","# We are #, # We are #",True,"01:21:12,224","01:21:15,321","01:21:12,814","01:21:15,510"
400,"...אנחנו, אנחנו","# We are, we are #",True,"01:21:19,879","01:21:22,489","01:21:20,355","01:21:22,949"


## Shift